<h1> IBM Applied Data Science Capstone Course by Coursera <h1>
<h3> Week 5 Final report <h3>
<em>Opening a New Pizza Shop in Mumbai , India <em>
<ul>
    <li>Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page</li>
    <li>Get the geographical coordinates of the neighborhoods</li>
    <li>Obtain the venue data for the neighborhoods from Foursquare API</li>
    <li>Explore and cluster the neighborhoods</li>
    <li>Select the best cluster to open a new shopping mall</li>
</ul>

<h3>1.) Import Libraries <h3>

In [2]:
!pip install geopy
!pip install geocoder
!pip install BeautifulSoup4
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 102kB 6.4MB/s ta 0:00:011
     |████████████████████████████████| 102kB 7.9MB/s ta 0:00:011
     |████████████████████████████████| 829kB 18.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 18.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
     |████████████████████████████████| 112kB 22.2MB/s eta 0:00:01
Libraries imported.


<h3>2.) Scrap data from Wikipedia page into a DataFrame</h3>

In [3]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
neighborhoodList = []

for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
mu_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mu_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [7]:
mu_df=mu_df.drop(mu_df.index[11])
mu_df=mu_df.drop(mu_df.index[39]) #Cleaning the dataframe

In [8]:
mu_df

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup
5,Borivali
6,Charkop
7,Chembur
8,Dahisar
9,Devipada


In [9]:
# print the number of rows of the dataframe
mu_df.shape

(40, 1)

<h3>3. Get the geographical coordinates</h3>

In [10]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
coords = [ get_latlng(neighborhood) for neighborhood in mu_df["Neighborhood"].tolist() ]

In [13]:
coords

[[19.118459378296492, 72.84176321065843],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.06218000000007, 72.90241000000003],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.08652321008152, 72.90900774216628],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21198153260436, 72.83757275783374],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21198153260436, 72.83757275783374],
 [19.048530000000028, 72.93222000000003],
 [19.044919875382305, 72.8672046443245

In [14]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [15]:
# merge the coordinates into the original dataframe
mu_df['Latitude'] = df_coords['Latitude']
mu_df['Longitude'] = df_coords['Longitude']


In [16]:
# check the neighborhoods and the coordinates
print(mu_df.shape)
mu_df

(40, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [17]:
#updating the NaN values
mu_df.at[41,'Latitude']=18.9986
mu_df.at[41,'Longitude']=72.8174
mu_df

,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [18]:
# save the DataFrame as CSV file
mu_df.to_csv("kl_df.csv", index=False)

<h3>4.) Create a map of Mumbai with neighborhoods superimposed on top</h3>

In [19]:
# get the coordinates of India
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [20]:
# create map of Mumbai using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mu_df['Latitude'], mu_df['Longitude'], mu_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [21]:
map_kl.save('map_kl.html')

<h3>5.) Use the Foursquare API to explore the neighborhoods</h3>

In [22]:
CLIENT_ID = 'K4UF3O23W4QQ3VWUN2PW2USBQVZNBYDPOEB55ECFKRI5O41Z'
CLIENT_SECRET = '1QDKWI3050R551K4I1OS202SMFLZ3WKMKSVPCRLUUVD02YTW' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K4UF3O23W4QQ3VWUN2PW2USBQVZNBYDPOEB55ECFKRI5O41Z
CLIENT_SECRET:1QDKWI3050R551K4I1OS202SMFLZ3WKMKSVPCRLUUVD02YTW


<h4>    Getting the top 100 venues that are within a radius of 2000 meters.</h4>

In [23]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mu_df['Latitude'], mu_df['Longitude'], mu_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [24]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2573, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118459,72.841763,Narayan Sandwich,19.121398,72.850270,Sandwich Place
4,Andheri,19.118459,72.841763,Temptations,19.113767,72.841337,Ice Cream Shop


In [25]:
venues_df.groupby(["Neighborhood"]).count() # checking how many venues were returned for each neighorhood

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,21,21,21,21,21,21
Baiganwadi,10,10,10,10,10,10
Bandra,100,100,100,100,100,100
Bhandup,25,25,25,25,25,25
Borivali,100,100,100,100,100,100
Charkop,54,54,54,54,54,54
Chembur,98,98,98,98,98,98
Dahisar,59,59,59,59,59,59


In [26]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
venues_df['VenueCategory'].unique()[:50]

There are 193 uniques categories.


array(['Bakery', 'Indian Restaurant', 'Ice Cream Shop', 'Sandwich Place',
       'Pub', 'Falafel Restaurant', 'Coffee Shop', 'Juice Bar',
       'Pizza Place', 'Multiplex', 'Snack Place', 'Seafood Restaurant',
       'Fast Food Restaurant', 'Chinese Restaurant', 'Breakfast Spot',
       'Café', 'American Restaurant', 'Maharashtrian Restaurant',
       'Cocktail Bar', 'Diner', 'Bar', 'Electronics Store', 'Spa',
       'Gym / Fitness Center', 'Asian Restaurant', 'BBQ Joint', 'Lounge',
       "Women's Store", 'Residential Building (Apartment / Condo)',
       'Smoke Shop', 'Food Truck', 'Vegetarian / Vegan Restaurant',
       'Liquor Store', 'Park', 'Athletics & Sports', 'Fish Market',
       'Martial Arts Dojo', 'Tea Room', 'Burger Joint', 'Food', 'Plaza',
       'Bus Station', 'Sports Bar', 'Gym', 'Platform',
       'Paper / Office Supplies Store', 'Food & Drink Shop',
       'Music Venue', 'Tunnel', 'Fried Chicken Joint'], dtype=object)

In [27]:
"Pizza Place" in venues_df['VenueCategory'].unique()

True

<h3> 6.) Analyze each suburb <h3>

In [28]:
# one hot encoding
mu_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mu_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mu_onehot.columns[-1]] + list(mu_onehot.columns[:-1])
mu_onehot = mu_onehot[fixed_columns]

print(mu_onehot.shape)
mu_onehot.head()

(2573, 194)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Station,Cafeteria,Café,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

<h4>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</h4>

In [29]:
mu_grouped = mu_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mu_grouped.shape)
mu_grouped

(39, 194)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Station,Cafeteria,Café,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0.00,0.000000,0.00,0.010000,0.00,0.000000,0.000000,0.020000,0.010000,0.010000,0.00,0.020000,0.000000,0.050000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.00,0.010000,0.000000,0.000000,0.040000,0.00,0.00,0.050000,0.000000,0.02,0.010000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.010000,0.000000,0.00,0.010000,0.00,0.000000,0.020000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.040000,0.180000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.020000,0.010000,0.000000,0.01,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.040

In [30]:
len(mu_grouped[mu_grouped["Pizza Place"] > 0])

33

<h4>Creating a new DataFrame for Pizza place data only</h4>

In [31]:
mu_pizza = mu_grouped[["Neighborhoods","Pizza Place"]]

In [32]:
mu_pizza.head()

,Neighborhoods,Pizza Place
0,Andheri,0.05
1,Anushakti Nagar,0.00
2,Baiganwadi,0.00
3,Bandra,0.03
4,Bhandup,0.08


<h3>7) Cluster Neighborhoods</h3>

<h4>Run k-means to cluster the neighborhoods in Mumbai into 3 clusters.</h4>

In [33]:
# set number of clusters
kclusters = 3

mu_clustering = mu_pizza.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mu_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

In [34]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mu_merged = mu_pizza.copy()

# add clustering labels
mu_merged["Cluster Labels"] = kmeans.labels_

In [35]:
mu_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mu_merged.head()

,Neighborhood,Pizza Place,Cluster Labels
0,Andheri,0.05,0
1,Anushakti Nagar,0.00,1
2,Baiganwadi,0.00,1
3,Bandra,0.03,0
4,Bhandup,0.08,0


In [36]:
mu_merged = mu_merged.join(mu_df.set_index("Neighborhood"), on="Neighborhood")

print(mu_merged.shape)
mu_merged.head() # check the last columns!

(39, 5)


,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
0,Andheri,0.05,0,19.118459,72.841763
1,Anushakti Nagar,0.00,1,19.042830,72.927340
2,Baiganwadi,0.00,1,19.062940,72.926630
3,Bandra,0.03,0,19.054370,72.840170
4,Bhandup,0.08,0,19.145560,72.948560


In [37]:
# sort the results by Cluster Labels
print(mu_merged.shape)
mu_merged.sort_values(["Cluster Labels"], inplace=True)
mu_merged

(39, 5)


,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
0,Andheri,0.050000,0,19.118459,72.841763
35,Vashi,0.030000,0,19.111090,72.927810
34,Uttan,0.060000,0,19.084650,72.904810
32,Thakur village,0.052632,0,18.996160,72.852790
31,"Sonapur, Bhandup",0.066667,0,19.210200,72.875410
30,"Sion, Mumbai",0.045455,0,19.163940,72.935440
29,Shil Phata,0.046875,0,19.043590,72.864120
27,Pestom sagar,0.030000,0,19.131342,72.816342
26,Mumbra,0.060241,0,19.070640,72.902170
24,Mogra Village,0.065789,0,19.171830,72.955650


<h4> Visualizing the resulting clusters</h4>

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mu_merged['Latitude'], mu_merged['Longitude'], mu_merged['Neighborhood'], mu_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [51]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<h3>8) Examine Clusters</h3>

<h4>Cluster 0</h4>

In [54]:
mu_merged.loc[mu_merged['Cluster Labels'] == 0]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
0,Andheri,0.050000,0,19.118459,72.841763
35,Vashi,0.040000,0,19.111090,72.927810
34,Uttan,0.060000,0,19.084650,72.904810
32,Thakur village,0.055556,0,18.996160,72.852790
31,"Sonapur, Bhandup",0.067797,0,19.210200,72.875410
30,"Sion, Mumbai",0.048780,0,19.163940,72.935440
29,Shil Phata,0.058824,0,19.043590,72.864120
26,Mumbra,0.058824,0,19.070640,72.902170
24,Mogra Village,0.063291,0,19.171830,72.955650
22,Mankhurd,0.056604,0,19.044920,72.867205


<h4>Cluster 1</h4>

In [55]:
mu_merged.loc[mu_merged['Cluster Labels'] == 1]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
36,Vikhroli,0.020833,1,19.017200,72.858170
1,Anushakti Nagar,0.000000,1,19.042830,72.927340
2,Baiganwadi,0.000000,1,19.062940,72.926630
33,Tilak Nagar (Mumbai),0.000000,1,26.866340,80.938840
3,Bandra,0.030000,1,19.054370,72.840170
5,Borivali,0.020408,1,19.229360,72.857510
28,Seven Bungalows,0.000000,1,19.146580,73.040050
14,Jogeshwari,0.030000,1,19.014920,72.845220
25,Mulund,0.000000,1,19.190540,73.022660
21,Mahavir Nagar (Kandivali),0.000000,1,19.048530,72.932220


<h4>Cluster 2</h4>

In [56]:
mu_merged.loc[mu_merged['Cluster Labels'] == 2]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
23,"Matharpacady, Mumbai",0.111111,2,19.280032,72.867932
4,Bhandup,0.086957,2,19.145560,72.948560
10,Dombivli,0.100000,2,19.212750,73.083240


<h4>Observations</h4>

Most of the Pizza Places are concentrated in the central area of Mumbai city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no Pizza shops in the neighborhoods. This represents a great opportunity and high potential areas to open new Pizza Joints as there is very little to no competition from existing Pizza shops. Meanwhile, Pizza shops in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of Pizza Shops. From another perspective, this also shows that the oversupply of Pizza Shops mostly happened in the central area of the city as major chunk of population resides there. Therefore, this project recommends property developers to capitalize on these findings to open new Pizza shops in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new Pizza shops in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of Pizza shops and suffering from intense competition.